In [ ]:
#!pip install yfinance --upgrade --no-cache-dir

In [1]:
import pickle
import numpy as np
import pandas as pd
import yfinance as yf 
import time
import datetime
import sqlite3

### Extract

In [3]:
cohist = pd.read_pickle("./constituents_history.pkl")
cohist.head(10)

,0
2021-12-31,[]
2021-11-30,[]
2021-10-29,[]
2021-09-30,[]
2021-08-31,[]
2021-07-30,"[[AAPL, APPLE INC, Information Technology, Equ..."
2021-06-30,"[[AAPL, APPLE INC, Information Technology, Equ..."
2021-05-28,"[[AAPL, APPLE INC, Information Technology, Equ..."
2021-04-30,"[[AAPL, APPLE INC, Information Technology, Equ..."
2021-03-31,"[[AAPL, APPLE INC, Information Technology, Equ..."


In [4]:
def extract(ticker):
    df = yf.download(ticker, start="2018-01-01")
    df.reset_index(inplace=True)
    return df

### Transform & Load

In [1]:
{
    "tags": [
        "hide-output"
    ]
}
# Create connection
con = sqlite3.connect('vanguardcons.db')
cur = con.cursor()

# Drop table if exists
cur.execute('''DROP TABLE IF EXISTS stocks;''')

# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS stocks (
               Date text, 
               ticker text, 
               Sector text,
               Open real, 
               High real, 
               Low real,
               Close real,
               AdjClose real,
               Volume real);''')

# Insert data
tmp_storage = {}
delisted = [] # delisted companies or no data after 2018-01-01
    
for i in range(len(cohist)):
    month_end = cohist.index[i]
    year = month_end.year
    month = month_end.month

    for company in cohist.iloc[i,0]:
        ticker = company[0]
        sector = company[2]

        if ticker not in tmp_storage.keys():
            df = extract(ticker)

            if len(df) == 0:
                delisted.append(ticker)
            tmp_storage[ticker] = df

        for j in range(1, month_end.day+1):
            df = tmp_storage[ticker]
            curr_date = datetime.datetime(year=year, month=month, day=j)
            curr_row = df[df["Date"]==pd.Timestamp(curr_date)]

            if len(curr_row) != 0:
                cr = np.array(curr_row)[0]
                params = (cr[0].strftime('%Y-%m-%d'), ticker, sector, cr[1], cr[2], cr[3], cr[4], cr[5], cr[6])
                SQL_command = """INSERT INTO stocks VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""
                cur.execute(SQL_command, params)

# Save (commit) the changes
con.commit()
con.close()


In [57]:
# delisted companies or no data after 2018-01-01
delisted

['BRKB',
 'XTSLA',
 'MXIM',
 'LB',
 'BFB',
 'PFPT',
 'HEIA',
 'WFFUT',
 'LENB',
 'ESU1',
 'ALXN',
 'WORK',
 'PRAH',
 'NAV',
 'TCF',
 'CLGX',
 'ESM1',
 'FLIR',
 'PRSP',
 'VAR',
 'IPHI',
 'RP',
 'PS',
 'EV',
 'ESH1',
 'TIF',
 'CXO',
 'AAXN',
 'PE',
 'WPX',
 '-',
 'GLIBA',
 'HDS',
 'DNKN',
 'TCO',
 'ESZ0',
 'MYL',
 'IMMU',
 'AMTD',
 'ETFC',
 'SERV',
 'NBL',
 'CTL',
 'LOGM',
 'WYND',
 'STAY',
 'WRI',
 'MOGA',
 'CMD',
 'JWA',
 'CLNY',
 'LGFB',
 'LGFA',
 'ESU0',
 'LM',
 'TECD',
 'EE',
 'POL',
 'IBKC',
 'PTLA',
 'TMUSR',
 'AXE',
 'APY',
 'LSXMR',
 'ESM0',
 'AGN',
 'WBC',
 'UTX',
 'RTN',
 'CY',
 'MSG',
 'ZAYO',
 'GDI',
 'ESH0',
 'WTR',
 'LPT',
 'WCG',
 'MDCO',
 'FII',
 'SNH',
 'BBT',
 'STI',
 'CBS',
 'JEC',
 'VIAB',
 'GWR',
 'DISHR',
 'ESZ9',
 'CELG',
 'HCP',
 'SYMC',
 'BHGE',
 'VSM',
 'MDSO',
 'BID',
 'TSS',
 'HPT',
 'TRCO',
 'FNSR',
 'SFLY',
 'ATU',
 'DLPH',
 'PLT',
 'QEP',
 'SEMG',
 'MIK',
 'XON',
 'AKRX',
 'MNK',
 'ESU9',
 'APC',
 'DATA',
 'TMK',
 'CHFC',
 'UBNT',
 'WP',
 'RHT',
 'HRS',
 '

In [17]:
# A preview of the stocks table from the database 
con = sqlite3.connect('vanguardcons.db')
cur = con.cursor()
cur.execute('''SELECT * FROM stocks LIMIT 10;''')
rows = cur.fetchall()
con.close()
print(rows)

[('2021-07-01', 'AAPL', 'Information Technology', 136.60000610351562, 137.3300018310547, 135.75999450683594, 137.27000427246094, 136.86489868164062, 52485800.0), ('2021-07-02', 'AAPL', 'Information Technology', 137.89999389648438, 140.0, 137.75, 139.9600067138672, 139.54696655273438, 78852600.0), ('2021-07-06', 'AAPL', 'Information Technology', 140.07000732421875, 143.14999389648438, 140.07000732421875, 142.02000427246094, 141.6008758544922, 108181800.0), ('2021-07-07', 'AAPL', 'Information Technology', 143.5399932861328, 144.88999938964844, 142.66000366210938, 144.57000732421875, 144.1433563232422, 104911600.0), ('2021-07-08', 'AAPL', 'Information Technology', 141.5800018310547, 144.05999755859375, 140.6699981689453, 143.24000549316406, 142.81727600097656, 105575500.0), ('2021-07-09', 'AAPL', 'Information Technology', 142.75, 145.64999389648438, 142.64999389648438, 145.11000061035156, 144.6817626953125, 99890800.0), ('2021-07-12', 'AAPL', 'Information Technology', 146.2100067138672, 1

### Analyze

In [35]:
# the index value (price) for every day that you have data in 2018, 2019, and 2020
con = sqlite3.connect('vanguardcons.db')
con.execute("""DROP TABLE IF EXISTS indextbl""") 
con.execute("""CREATE TABLE indextbl AS 
                SELECT Date, avg(Close) AS DailyIndex 
                FROM stocks
                GROUP BY Date""")
cur = con.cursor()
cur.execute('''SELECT * FROM indextbl limit 20;''')
indices = cur.fetchall()
con.commit()
con.close()
print(indices)

[('2018-01-01', None), ('2018-01-02', 86.02972694285516), ('2018-01-03', 86.57170976224559), ('2018-01-04', 86.783789523069), ('2018-01-05', 87.32400225040462), ('2018-01-08', 87.64131091754774), ('2018-01-09', 87.68282397748503), ('2018-01-10', 87.28583643504183), ('2018-01-11', 88.18935998802172), ('2018-01-12', 88.67461744823596), ('2018-01-16', 88.19073227776093), ('2018-01-17', 89.01853848915684), ('2018-01-18', 88.76914080315753), ('2018-01-19', 89.56901193176932), ('2018-01-22', 90.06518070876416), ('2018-01-23', 90.32778454563169), ('2018-01-24', 90.23467290687451), ('2018-01-25', 90.15943931704132), ('2018-01-26', 90.75320572986482), ('2018-01-29', 90.01893794686798)]


In [56]:
# the sector distribution of the index for 2018, 2019, and 2020 on a relative weight basis (at year end)
con = sqlite3.connect('vanguardcons.db')
cur = con.cursor()
cur.execute("""SELECT strftime('%Y',Date), Sector, ticker, Close 
                FROM stocks
                WHERE strftime('%m',Date) = '12' AND strftime('%d',Date) = '31'""")
sector_info_lst = cur.fetchall()
con.close()

# Store sector info in a dictionary
sector_info = {} # year -> sector -> ticker -> value  

for row in sector_info_lst:
    year, sector, ticker, close = row
    if year not in sector_info:
        sector_info[year] = {}
    if sector not in sector_info[year]:
        sector_info[year][sector] = {}
    sector_info[year][sector][ticker] = close
    
# Calculate relative weight of Close price index for each sector 
lst_of_lists = [] 

for year, value in sector_info.items():
    for sector, value2 in value.items():
        tot = 0
        for ticker, value3 in value2.items():
            tot += value3
        for ticker, value3 in value2.items():
            lst = [year, sector, ticker, value3/tot]
            lst_of_lists.append(lst)
            
sector_dist = pd.DataFrame(lst_of_lists)
sector_dist.columns = ['Year', 'Sector', 'Ticker', 'RelWeight']
sector_dist

,Year,Sector,Ticker,RelWeight
0,2020,Information Technology,AAPL,0.005213
1,2020,Information Technology,MSFT,0.008738
2,2020,Information Technology,V,0.008593
3,2020,Information Technology,NVDA,0.005129
4,2020,Information Technology,MA,0.014023
...,...,...,...,...
3229,2018,Real Estate,DRH,0.001723
3230,2018,Real Estate,UE,0.003153
3231,2018,Real Estate,LXP,0.001558
3232,2018,Real Estate,SITC,0.002100
